## This is a notebook for practicing the optimization portion of prohack problem

In [ ]:
import pandas as pd
import numpy as np

# Insert code here.
t_df=pd.read_csv('/home/jovyan/work/data/results_df.csv')
t_df.groupby('galaxy').apply(lambda grp: grp['galaxy'].sort_values().tail(1))

galaxy                                   
Andromeda Galaxy (M31)                394                  Andromeda Galaxy (M31)
Andromeda I                           328                             Andromeda I
Andromeda II                          857                            Andromeda II
Andromeda III                         811                           Andromeda III
Andromeda IX                          864                            Andromeda IX
                                                             ...                 
Ursa Major II Dwarf                   869                     Ursa Major II Dwarf
Ursa Minor Dwarf                      679                        Ursa Minor Dwarf
Virgo I                               67                                  Virgo I
Willman 1                             749                               Willman 1
Wolf-Lundmark-Melotte (WLM, DDO 221)  152    Wolf-Lundmark-Melotte (WLM, DDO 221)
Name: galaxy, Length: 172, dtype: object

In [ ]:
last_point_in_time_df = t_df.groupby('galaxy').apply(lambda grp: grp[['galactic year', 'existence expectancy index','y_pred']].sort_values('galactic year').tail(1))
last_point_in_time_df[last_point_in_time_df['existence expectancy index'] < 0.7]

,,galactic year,existence expectancy index,y_pred
galaxy,,,,
Andromeda VIII,41,1016064,0.659660,0.026155
Andromeda XVII,58,1014049,0.684616,0.021320
Antlia B,45,1011030,0.662105,0.030149
Hydra II,57,1016064,0.682815,0.023656
KKH 12,52,1013042,0.677107,0.023490
KKR 25,17,1011030,0.623630,0.030818
Pegasus III,49,1015056,0.671839,0.015685
Reticulum III,34,1015056,0.650696,0.019548
UGC 8651 (DDO 181),56,1013042,0.682547,0.024611


In [ ]:
last_point_in_time_df['in_need'] = last_point_in_time_df['existence expectancy index']<0.7

In [ ]:
print(len(in_need_df), 'in need')
print(len(not_in_need_df), 'not in need')
print('we therefor use',(len(in_need_df)+len(not_in_need_df))*100 )

NameError: name 'in_need_df' is not defined

1. pass in df with galaxies, existence exp and predicted index
2. rank order all the galaxies by asceding predicted index, thus this holds for future allocations
3. how many galaxies have existence expectancy < 0.7 : call this m. 
4. give the first min(m,50) of the above galaxies 100 each and remove them from the df
5. in the remaining df, give the first m * 9 galaxies 100 each

In [ ]:
last_point_in_time_df['potential_for_increase_index']=-np.log(last_point_in_time_df['y_pred']+0.01)+3
t_df['potential_for_increase_index']=-np.log(t_df['y_pred']+0.01)+3

In [ ]:
def optimization(predicted_df):
    '''
    this function reads in the full dataframe with predictions already done. it must also contain galaxies
    '''
    existence_cut_off = 0.7
    predicted_df = predicted_df.sort_values('potential_for_increase_index', ascending = False)
    poor_people = predicted_df[predicted_df['existence expectancy index'] < existence_cut_off]
    rich_people = predicted_df[predicted_df['existence expectancy index'] >= existence_cut_off]

    m = min(len(poor_people), 50)
    poor_people = poor_people.head(m)
    rich_people = rich_people.head(m * 9)

    all_people = poor_people.append(rich_people)
    all_people['opt_pred'] = 100

    predicted_df = predicted_df.join(all_people, rsuffix="_right")
    predicted_df.loc[:,~predicted_df.columns.str.contains('_right', case=False)] 

    predicted_df['opt_pred'] = predicted_df['opt_pred'].fillna(0)
    return predicted_df
    
submit_df = optimization(t_df)
print(sum(submit_df['opt_pred']))

50000.0


In [ ]:
sum(submit_df['opt_pred']*submit_df['potential_for_increase_index']**2/1000)

1818.6860499967884

In [ ]:
#submit_df[['galactic year', 'y_pred', 'opt_pred']]#.to_csv('submit.csv')
sum(submit_df['opt_pred'])

50000.0

In [ ]:
submit_df['galaxyy'] = submit_df.index.gsuet_level_values(0)


new_df = pd.merge(t_df, submit_df[['opt_pred','galaxyy','galactic year']],  how='left', left_on=['galaxy','galactic year'], right_on = ['galaxy','galactic year'],indicator=True)


KeyError: 'galaxy'

In [ ]:
new_df['opt_pred'] = submit_df['opt_pred'].fillna(0)
new_df[['y_pred','opt_pred']].to_csv('submit.csv')